In [1]:
import os
import sys
import cv2
import numpy as np
from PIL import Image
import glob
from tqdm import tqdm
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets

ROOT = os.getcwd()
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

from metrics.losses import PatchLoss
from dataset.FAS_dataset import FASDataset
from utils.utils import read_cfg, build_network, get_device

In [2]:
#c:/Users/alitu/Desktop/biometric/Implementation-patchnet/fas_1/Scripts/python.exe -m pip install ipykernel -U --force-reinstall

In [3]:
cfg = read_cfg(cfg_file='config/config.yaml')
device = get_device(cfg)

network = build_network(cfg, device)
network = network.to(device)
loss = PatchLoss().to(device)
saved_name = "C:\\Users\\alitu\\Desktop\\biometric\\Implementation-patchnet\\runs\\save\\swin_base_99_3.701748856476375.pth"
state = torch.load(saved_name)

network.load_state_dict(state['state_dict'])
loss.load_state_dict(state["loss"])
print("loading done!")
network.eval()


c:\Users\alitu\Desktop\biometric\Implementation-patchnet\fas_1\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\alitu\Desktop\biometric\Implementation-patchnet\fas_1\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


loading done!


FeatureExtractor(
  (nets): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats

In [14]:
transform = transforms.Compose([

   # transforms.ToPILImage(),
    transforms.Resize(cfg['model']['image_size']),
    transforms.RandomCrop(cfg['dataset']['augmentation']['rand_crop_size']),
    transforms.ToTensor(),
    transforms.Normalize(cfg['dataset']['mean'], cfg['dataset']['sigma'])

])

In [21]:
df_test = pd.read_csv("C:\\Users\\alitu\\Desktop\\biometric\\Implementation-patchnet\\datasets\\test.csv")

In [22]:
df_test['img_path'] = df_test['image_name'].apply(lambda x:'C:/Users/alitu/Desktop/biometric/Implementation-patchnet/datasets/images/' + x )

In [23]:
df_test

,Unnamed: 0,image_name,label,img_path
0,0,frame0vid1_1_11_1.jpg,1.0,C:/Users/alitu/Desktop/biometric/Implementatio...
1,1,frame0vid1_1_12_1.jpg,1.0,C:/Users/alitu/Desktop/biometric/Implementatio...
2,2,frame0vid1_1_13_1.jpg,1.0,C:/Users/alitu/Desktop/biometric/Implementatio...
3,3,frame0vid1_1_14_1.jpg,1.0,C:/Users/alitu/Desktop/biometric/Implementatio...
4,4,frame0vid1_1_15_1.jpg,1.0,C:/Users/alitu/Desktop/biometric/Implementatio...
...,...,...,...,...
99,47,frame0vid2_3_13_3.jpg,0.0,C:/Users/alitu/Desktop/biometric/Implementatio...
100,48,frame0vid2_3_13_4.jpg,0.0,C:/Users/alitu/Desktop/biometric/Implementatio...
101,49,frame0vid3_1_11_2.jpg,0.0,C:/Users/alitu/Desktop/biometric/Implementatio...
102,50,frame0vid3_1_11_4.jpg,0.0,C:/Users/alitu/Desktop/biometric/Implementatio...


In [24]:
result_path = "test_result_patchnet.csv"
d = {"img_path":[], "liveness_score":[]}

In [25]:
live_counter = 0
spoof_counter = 0

In [28]:
df_test['score'] = None
df_test['result'] = None

In [29]:
for index, row in df_test.iterrows():
    print(row['img_path'])
    image = Image.open(row['img_path'])
    image = transform(image)
    image = image.unsqueeze(0)
    feature = network.forward(image)
    feature = F.normalize(feature, dim=1)
    row['score'] = F.softmax(loss.amsm_loss.s * loss.amsm_loss.fc(feature.squeeze()), dim=0)
    row['result'] = torch.argmax(row['score'] )



C:/Users/alitu/Desktop/biometric/Implementation-patchnet/datasets/images/frame0vid1_1_11_1.jpg
C:/Users/alitu/Desktop/biometric/Implementation-patchnet/datasets/images/frame0vid1_1_12_1.jpg
C:/Users/alitu/Desktop/biometric/Implementation-patchnet/datasets/images/frame0vid1_1_13_1.jpg
C:/Users/alitu/Desktop/biometric/Implementation-patchnet/datasets/images/frame0vid1_1_14_1.jpg
C:/Users/alitu/Desktop/biometric/Implementation-patchnet/datasets/images/frame0vid1_1_15_1.jpg
C:/Users/alitu/Desktop/biometric/Implementation-patchnet/datasets/images/frame0vid1_1_16_1.jpg
C:/Users/alitu/Desktop/biometric/Implementation-patchnet/datasets/images/frame0vid1_1_17_1.jpg
C:/Users/alitu/Desktop/biometric/Implementation-patchnet/datasets/images/frame0vid1_2_11_1.jpg
C:/Users/alitu/Desktop/biometric/Implementation-patchnet/datasets/images/frame0vid1_2_12_1.jpg
C:/Users/alitu/Desktop/biometric/Implementation-patchnet/datasets/images/frame0vid1_2_13_1.jpg
C:/Users/alitu/Desktop/biometric/Implementation-pa

In [12]:
image = Image.open("C:/Users/alitu/Desktop/biometric/Implementation-patchnet/datasets/images/frame0vid1_1_11_1.jpg")

In [30]:
df_test[df_test['score'] == 1]

,Unnamed: 0,image_name,label,img_path,score,result


In [38]:
transform = transforms.Compose([

    #transforms.ToPILImage(),
    transforms.Resize(cfg['model']['image_size']),
    transforms.RandomCrop(cfg['dataset']['augmentation']['rand_crop_size']),
    transforms.ToTensor(),
    transforms.Normalize(cfg['dataset']['mean'], cfg['dataset']['sigma'])

])

In [52]:
def predict(src):
    image = Image.open(src)
    image = transform(image)
    image = image.unsqueeze(0)
    feature = network.forward(image)
    feature = F.normalize(feature, dim=1)
    score = F.softmax(loss.amsm_loss.s * loss.amsm_loss.fc(feature.squeeze()), dim=0)
    result = torch.argmax(score )

    return result



In [58]:
df_test['result'] = df_test['img_path'].apply(lambda x: predict(x).item())

In [54]:
df_test.iloc[0]

Unnamed: 0                                                    0
image_name                                frame0vid1_1_11_1.jpg
label                                                       1.0
img_path      C:/Users/alitu/Desktop/biometric/Implementatio...
score         [tensor(5.4445e-08, grad_fn=<UnbindBackward0>)...
result                                                tensor(1)
Name: 0, dtype: object

In [56]:
# df_test[df_test['label'] == 0.0]

,Unnamed: 0,image_name,label,img_path,score,result
52,0,frame0vid1_1_11_2.jpg,0.0,C:/Users/alitu/Desktop/biometric/Implementatio...,"[tensor(1.0000, grad_fn=<UnbindBackward0>), te...",tensor(0)
53,1,frame0vid1_1_11_3.jpg,0.0,C:/Users/alitu/Desktop/biometric/Implementatio...,"[tensor(0.9999, grad_fn=<UnbindBackward0>), te...",tensor(0)
54,2,frame0vid1_1_11_4.jpg,0.0,C:/Users/alitu/Desktop/biometric/Implementatio...,"[tensor(1.0000, grad_fn=<UnbindBackward0>), te...",tensor(0)
55,3,frame0vid1_1_11_5.jpg,0.0,C:/Users/alitu/Desktop/biometric/Implementatio...,"[tensor(1.0000, grad_fn=<UnbindBackward0>), te...",tensor(0)
56,4,frame0vid1_1_12_2.jpg,0.0,C:/Users/alitu/Desktop/biometric/Implementatio...,"[tensor(1.0000, grad_fn=<UnbindBackward0>), te...",tensor(0)
57,5,frame0vid1_1_12_3.jpg,0.0,C:/Users/alitu/Desktop/biometric/Implementatio...,"[tensor(1.0000, grad_fn=<UnbindBackward0>), te...",tensor(0)
58,6,frame0vid1_1_12_4.jpg,0.0,C:/Users/alitu/Desktop/biometric/Implementatio...,"[tensor(1.0000, grad_fn=<UnbindBackward0>), te...",tensor(0)
59,7,frame0vid1_1_12_5.jpg,0.0,C:/Users/alitu/Desktop/biometric/Implementatio...,"[tensor(1.0000, grad_fn=<UnbindBackward0>), te...",tensor(0)
60,8,frame0vid1_1_13_2.jpg,0.0,C:/Users/alitu/Desktop/biometric/Implementatio...,"[tensor(1.0000, grad_fn=<UnbindBackward0>), te...",tensor(0)
61,9,frame0vid1_1_13_3.jpg,0.0,C:/Users/alitu/Desktop/biometric/Implementatio...,"[tensor(1.0000, grad_fn=<UnbindBackward0>), te...",tensor(0)


In [62]:
df_test.drop(['Unnamed: 0'] , axis = 1)

,image_name,label,img_path,score,result
0,frame0vid1_1_11_1.jpg,1.0,C:/Users/alitu/Desktop/biometric/Implementatio...,"[tensor(5.4445e-08, grad_fn=<UnbindBackward0>)...",1
1,frame0vid1_1_12_1.jpg,1.0,C:/Users/alitu/Desktop/biometric/Implementatio...,"[tensor(4.7269e-08, grad_fn=<UnbindBackward0>)...",1
2,frame0vid1_1_13_1.jpg,1.0,C:/Users/alitu/Desktop/biometric/Implementatio...,"[tensor(5.9049e-08, grad_fn=<UnbindBackward0>)...",1
3,frame0vid1_1_14_1.jpg,1.0,C:/Users/alitu/Desktop/biometric/Implementatio...,"[tensor(1.1294e-07, grad_fn=<UnbindBackward0>)...",1
4,frame0vid1_1_15_1.jpg,1.0,C:/Users/alitu/Desktop/biometric/Implementatio...,"[tensor(7.5904e-08, grad_fn=<UnbindBackward0>)...",1
...,...,...,...,...,...
99,frame0vid2_3_13_3.jpg,0.0,C:/Users/alitu/Desktop/biometric/Implementatio...,"[tensor(0.9999, grad_fn=<UnbindBackward0>), te...",0
100,frame0vid2_3_13_4.jpg,0.0,C:/Users/alitu/Desktop/biometric/Implementatio...,"[tensor(1.0000, grad_fn=<UnbindBackward0>), te...",0
101,frame0vid3_1_11_2.jpg,0.0,C:/Users/alitu/Desktop/biometric/Implementatio...,"[tensor(1.0000, grad_fn=<UnbindBackward0>), te...",0
102,frame0vid3_1_11_4.jpg,0.0,C:/Users/alitu/Desktop/biometric/Implementatio...,"[tensor(1.0000, grad_fn=<UnbindBackward0>), te...",0


In [64]:
df_test.to_csv('test_patchnet_result.csv')

In [65]:
y_test = df_test['label'] 
y_pred = df_test['result'] 

In [66]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score

# Assuming you have a trained classification model 'clf'
# Make predictions on the test set


# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Generate a classification report
classification_rep = classification_report(y_test, y_pred)
print("Classification Report:")
print(classification_rep)

# Create a confusion matrix
confusion_mat = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(confusion_mat)


Accuracy: 0.93
Classification Report:
              precision    recall  f1-score   support

         0.0       0.96      0.90      0.93        52
         1.0       0.91      0.96      0.93        52

    accuracy                           0.93       104
   macro avg       0.93      0.93      0.93       104
weighted avg       0.93      0.93      0.93       104

Confusion Matrix:
[[47  5]
 [ 2 50]]
